## Shrinkage methods for linear regression models 

In this lesson we are going to learn how to:

1. Estimate a ridge regression model with fixed penalty term
2. Estimate a lasso regression model with fixed penalty term
3. Cross-validate the penalty terms both for the ridge regression and the lasso
4. Compare a simple linear regression model against both the ridge and the lasso regression models

As usual, the first step is to import the data by using the **pandas** package. The data refers to median house prices in a metropolitan area in the US. In addition to the target variable, median house price for a given are, we also have a set of predictors such as the portion of the house built before 1940 (**age**), number of rooms (**rm**), per capita crime rate (**crim**) and tax rate (**tax**). 

In [3]:
import pandas as pd

df = pd.read_csv('Boston.csv',index_col = 0)

### Estimate a linear regression model

As a first step, we are going to estimate a linear regression model. This is going to be our baseline model against which we compare the ridge and the lasso. To estimate the linear regression we are going to use the ***sklearn*** package. Since we are interested in forecasting, and ultimately to assess the performance of a linear regression model out of sample, we need to split the train vs testing observations. This is done using the ***train_test_split*** function. 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define and standardise the predictors
X       = df[['rm','nox','indus','crim','tax','ptratio','lstat','age','dis']]
X       = StandardScaler().fit_transform(X)

# Define and standardise the target variables
y       = df['medv'].values.reshape(-1,1)
y       = StandardScaler().fit_transform(y)

# Split the whole data in train vs test data. We take a third of the data as testing sample. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.linear_model import LinearRegression

# Define and estimate the model
lr       = LinearRegression(fit_intercept = False).fit(X_train, y_train)

ypred_lr = lr.predict(X_test)

Let's calcolate now the forecasting performance based on the $R^2$ coefficient. 

In [20]:
from sklearn.metrics import r2_score

r2_lr = round(r2_score(y_test, ypred_lr),2)

accuracy = {'ols': {'r2 ': r2_lr}} #creating a dictionary
print(accuracy)

{'ols': {'r2 ': 0.7}}


### Estimate a ridge regression model

We now estimate a ridge regression model with a fixed penalty parameter. Notice in the python syntax, the penalty term $\lambda$ is called ***alpha***

In [11]:
from sklearn.linear_model import Ridge

alpha = 10 # This is the penalty term
ridge = Ridge(alpha = alpha, fit_intercept = False) # Normalise means that the predictors are 
ridge.fit(X_train,y_train)

ypred_ridge = ridge.predict(X_test)

In [21]:
r2_ridge = round(r2_score(y_test, ypred_ridge),2)

accuracy['ridge'] = r2_ridge
print(accuracy)

{'ols': {'r2 ': 0.7}, 'ridge': 0.7}


### Estimate a lasso regression

We now estimate a ridge regression model with a fixed penalty parameter. Notice in the python syntax, the penalty term $\lambda$ is called ***alpha***. For simplicity, we use the same penalty term used for the ridge regression. 

In [22]:
from sklearn.linear_model import Lasso

alpha = 0.1 # This is the penalty term
lasso = Lasso(alpha = alpha, fit_intercept = False) 
lasso.fit(X_train,y_train)

ypred_lasso = lasso.predict(X_test)

r2_lasso = round(r2_score(y_test, ypred_lasso),2)

accuracy['lasso'] = r2_lasso
print(accuracy)

{'ols': {'r2 ': 0.7}, 'ridge': 0.7, 'lasso': 0.65}


We can now collect the results across models and compare the performances. 

In [23]:
accuracy = pd.DataFrame(accuracy)
print(accuracy)

     ols  ridge  lasso
r2   0.7    0.7   0.65


For this level of shrinkage, $\lambda=10$ the OLS seems to perform better than the others. We now try to estimate the penalty terms, instead of fixing them, via cross-validation. 

### Cross-validation of the penalty terms 

Cross validation of the penalty terms can be implemented via the function ***GridSearchCV***, both for the ridge and for the lasso regression models. 

In [25]:
from sklearn.model_selection import GridSearchCV
import numpy as np

alpha_space    = np.linspace(1, 10, 20) # Here we explore a penalty term from 0.1 to 100
param_grid = {'alpha': alpha_space}

ridge_cv  = GridSearchCV(Ridge(fit_intercept = False), param_grid, cv = 5) 
#GridSearchCV use dictionary as argument

We can now fit the model and produce the corresponding forecasts

In [26]:
ridge_cv.fit(X_train, y_train)

ypred_ridge_cv = ridge_cv.predict(X_test)

r2_ridge_cv = round(r2_score(y_test, ypred_ridge_cv),2)

accuracy['ridge cv'] = r2_ridge_cv

We can use the same procedure for the lasso. 

In [27]:
alpha_space    = np.linspace(0.1, 2, 20) # Here we explore a penalty term from 0.1 to 2
param_grid = {'alpha': alpha_space}

lasso_cv  = GridSearchCV(Lasso(fit_intercept = False), param_grid, cv = 5) 

In [28]:
lasso_cv.fit(X_train, y_train)

ypred_lasso_cv = lasso_cv.predict(X_test)

r2_lasso_cv = round(r2_score(y_test, ypred_lasso_cv),2)

accuracy['lasso cv'] = r2_lasso_cv

In [29]:
accuracy

,ols,ridge,lasso,ridge cv,lasso cv
r2,0.7,0.7,0.65,0.7,0.65


In [30]:
print('Penalty term for the ridge regression: \n', ridge_cv.best_estimator_)
print('Penalty term for the lasso regression: \n', lasso_cv.best_estimator_)

Penalty term for the ridge regression: 
 Ridge(alpha=10.0, fit_intercept=False)
Penalty term for the lasso regression: 
 Lasso(alpha=0.1, fit_intercept=False)
